In [2]:
import pandas as pd

# read in all data
test = pd.read_csv('../data/test.txt', delimiter=';', names=['text', 'target'])
train = pd.read_csv('../data/train.txt', delimiter=';',
                    names=['text', 'target'])
val = pd.read_csv('../data/val.txt', delimiter=';', names=['text', 'target'])
trainval = pd.concat([train,val])
testval = pd.concat([test,val])

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

# create vectorier for BoW
vectorizer = CountVectorizer(max_df=0.9, min_df=5, stop_words='english', ngram_range=(1, 1))
BoW = vectorizer.fit_transform(trainval.text)
print('Number of Features in BoW: ', len(vectorizer.get_feature_names_out()))

enc = LabelEncoder().fit(['anger', 'fear', 'joy', 'love', 'sadness', 'surprise'])

# transform all the data
X_trainval = vectorizer.transform(trainval.text).toarray()
Y_trainval = enc.transform(trainval.target)
X_train = vectorizer.transform(train.text).toarray()
Y_train = enc.transform(train.target)
X_val = vectorizer.transform(val.text).toarray()
Y_val = enc.transform(val.target)
X_test = vectorizer.transform(test.text).toarray()
Y_test = enc.transform(test.target)

Number of Features in BoW:  3398


In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

clf = XGBClassifier(n_estimators=100, max_depth=6, random_state=0, eval_metric='mlogloss', use_label_encoder=False)

params = {'n_estimators':range(1,250, 10), 
          'max_depth':range(1,100,10)}

#grid_cv = GridSearchCV(clf,param_grid=params, scoring='f1_macro', verbose=3, n_jobs=1, cv=5)
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
rand_cv = RandomizedSearchCV(clf, param_distributions=params, n_iter=5, verbose=3, scoring='f1_macro', cv=skf.split(X_trainval, Y_trainval), random_state=0)
rand_cv.fit(X_trainval, Y_trainval)

print('Best Parameters: ', rand_cv.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


KeyboardInterrupt: 